In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

onet_soc_xwalk = pd.read_excel("https://www.onetcenter.org/taxonomy/2019/soc/2019_to_SOC_Crosswalk.xlsx?fmt=xlsx", skiprows=3)
onet_soc_xwalk.rename(
    columns= lambda x: x.replace(' ','_').replace('-','_').replace('*','').replace("2019_","").replace("2018_","").upper(), 
    inplace=True
    )
onet_soc_xwalk.rename(columns={'SOC_CODE':'OCC_CODE', 'SOC_TITLE':'OCC_TITLE'}, inplace=True)
onet_soc_xwalk

,ONET_SOC_CODE,ONET_SOC_TITLE,OCC_CODE,OCC_TITLE
0,11-1011.00,Chief Executives,11-1011,Chief Executives
1,11-1011.03,Chief Sustainability Officers,11-1011,Chief Executives
2,11-1021.00,General and Operations Managers,11-1021,General and Operations Managers
3,11-1031.00,Legislators,11-1031,Legislators
4,11-2011.00,Advertising and Promotions Managers,11-2011,Advertising and Promotions Managers
...,...,...,...,...
1011,55-3014.00,Artillery and Missile Crew Members,55-3014,Artillery and Missile Crew Members
1012,55-3015.00,Command and Control Center Specialists,55-3015,Command and Control Center Specialists
1013,55-3016.00,Infantry,55-3016,Infantry
1014,55-3018.00,Special Forces,55-3018,Special Forces


# ONET DATA

In [152]:
DATA_DIR_ONET = '../../../data/onet_data/processed/measure/'

# Load data
WorkActivity = pd.read_csv(os.path.join(DATA_DIR_ONET, 'WORK_ACTIVITIES.csv'))
WorkContext = pd.read_csv(os.path.join(DATA_DIR_ONET, 'WORK_CONTEXT.csv'))

In [153]:
WorkActivity = WorkActivity.loc[
    ( WorkActivity.SCALE_ID == "IM" ),
    ['ONET_SOC_CODE', 'ELEMENT_NAME', 'ELEMENT_ID', 'DATA_VALUE', 'N']
]
WorkActivity['ELEMENT_NAME'] = 'wa_' + WorkActivity['ELEMENT_NAME']

WorkContext = WorkContext.loc[
    ( WorkContext.SCALE_ID  == "CX" ),
    ['ONET_SOC_CODE', 'ELEMENT_NAME', 'ELEMENT_ID', 'DATA_VALUE', 'N']
]
WorkContext['ELEMENT_NAME'] = 'wc_' + WorkContext['ELEMENT_NAME']

In [141]:
WorkActivityContext = pd.concat([WorkActivity, WorkContext])
N = np.round( WorkActivityContext.groupby("ONET_SOC_CODE").N.mean() )
WorkActivityContext["ELEMENT_NAME"] = (
    WorkActivityContext["ELEMENT_NAME"]
    .str.replace(" ", "_", regex=False)
    .str.replace("/", "_", regex=False)
    .str.replace(",", "", regex=False)
    .str.replace("-", "_", regex=False)
    .str.replace("_with_", "_", regex=False)
)
WorkActivityContext["ELEMENT_NAME"] = WorkActivityContext["ELEMENT_NAME"].str[:20] + "_" + WorkActivityContext["ELEMENT_ID"]
WorkActivityContext["ELEMENT_NAME"] = WorkActivityContext["ELEMENT_NAME"].str.replace(".", "", regex=False)
WorkActivityContext["ELEMENT_NAME"] = WorkActivityContext["ELEMENT_NAME"].str.replace("__", "_", regex=False)
WorkActivityContext = WorkActivityContext.pivot(
    index='ONET_SOC_CODE', 
    columns=['ELEMENT_NAME'], 
    values='DATA_VALUE').rename_axis(None, axis=1)
WorkActivityContext["N"] = N
WorkActivityContext = WorkActivityContext.reset_index()

WorkActivityContext = WorkActivityContext.merge(
    onet_soc_xwalk[['ONET_SOC_CODE', 'OCC_CODE', 'OCC_TITLE']],
    on='ONET_SOC_CODE', 
    how='inner')


# select columns to compute the weighted mean (columns starting with 'wa_' or 'wc_')
weighted_cols = [col for col in WorkActivityContext.columns if col.startswith('wa_') or col.startswith('wc_')]

# build aggregation dictionary: weighted average using N as weights and first value for OES_TITLE
agg_dict = {col: (lambda x: np.average(x, weights=WorkActivityContext.loc[x.index, 'N'])) for col in weighted_cols}
agg_dict['OES_TITLE'] = 'first'

# group by OES_CODE (representing 'oes2019') and aggregate
# WorkActivityContext = WorkActivityContext.groupby('OES_CODE').agg(agg_dict).reset_index()
# reverse email so that larger number indicates less email (= harder to WFH)
WorkActivityContext['wc_Electronic_Mail_4C1a2h'] = 5 - WorkActivityContext['wc_Electronic_Mail_4C1a2h'] + 1

# BLS DATA

In [99]:
DATA_DIR_OES = '../../../data/processed/bls/oews/oews_all_2023.csv'
oes = pd.read_csv(DATA_DIR_OES, low_memory=False)

In [164]:
oes_nat = oes.loc[
    (oes.AREA == 99) &
    (oes.O_GROUP == 'detailed') &
    (oes.I_GROUP == "cross-industry"),
    ['OCC_CODE', "OCC_TITLE", "TOT_EMP"]
]
oes_nat["TOT_EMP"] = pd.to_numeric(oes_nat["TOT_EMP"], errors='coerce')
oes_nat = oes_nat.groupby(["OCC_CODE", "OCC_TITLE"]).TOT_EMP.sum().reset_index()

In [101]:
oes_nat_ind = oes.loc[
    (oes.AREA == 99) &
    (oes.I_GROUP == "4-digit") &
    (oes.O_GROUP == 'detailed'),
    ['OCC_CODE', "OCC_TITLE", "TOT_EMP"] + [col for col in oes.columns if col.startswith('NAICS')]
]
oes_nat_ind["TOT_EMP"] = pd.to_numeric(oes_nat_ind["TOT_EMP"], errors='coerce')
oes_nat_ind = oes_nat_ind.groupby(["OCC_CODE", "OCC_TITLE", "NAICS", "NAICS_TITLE"]).TOT_EMP.sum().reset_index()

# EXPOSURE MEASURES

In [208]:
# Merge the two datasets on the common key "oes2019"

merged = pd.merge(oes_nat[["OCC_CODE", "TOT_EMP"]], WorkActivityContext, on="OCC_CODE", how="outer", indicator=True)

# Count the OES occupations that did not merge (rows coming only from the OES file)
num_left_only = merged[merged['_merge'] == 'left_only'].shape[0]
print("Num. of OES occs which don't merge =", num_left_only)

# Tabulate the merge key for the OES observations that did not match
list_left_only = merged.loc[merged['_merge'] == 'left_only', 'OCC_CODE'].values
print(list_left_only)

# For OES obs that did not merge, compute the total employment (TOT_EMP) in millions
total_emp = oes_nat.TOT_EMP.sum() / 1_000_000
aux1_nat = merged.loc[merged['_merge'] == 'left_only', 'TOT_EMP'].sum() / 1_000_000
print(f"Emp. (millions) in national OES which doesn't have O*NET vars = {aux1_nat} ( {aux1_nat/total_emp:.2%} )")

# For merged observations (matched in both), compute total employment in millions  
list_both = merged.loc[merged['_merge'] == 'both', "OCC_CODE"].unique()
aux2_nat = oes_nat[oes_nat.OCC_CODE.isin(list_both)].TOT_EMP.sum() / 1_000_000
print(f"Emp. (millions) in nat OES which does have O*NET vars = {aux2_nat} ( {aux2_nat/total_emp:.2%} )")

merged.drop(columns=['_merge'], inplace=True)

cols = [col for col in merged.columns if col.startswith('wa_') or col.startswith('wc_')]
merged = merged.groupby(["OCC_CODE", "OCC_TITLE"]).apply(
    lambda x: pd.Series({
        **{c: np.average(x[c], weights=x['N']) for c in cols},
        **{'TOT_EMP': x['TOT_EMP'].sum()}
    }),include_groups=False).reset_index()

merged.rename(columns={"wc_Physical_Proximit_4C2a3" : "pp"}, inplace=True)
merged["wc_Electronic_Mail_4C1a2h"] = 5 - merged["wc_Electronic_Mail_4C1a2h"] + 1
cols.remove("wc_Physical_Proximit_4C2a3")
# Convert to binary
merged[cols] = (merged[cols] > 3.5).astype(int)
merged.set_index(["OCC_CODE", "OCC_TITLE"], inplace=True)

Num. of OES occs which don't merge = 89
['11-1031' '11-2032' '11-2033' '11-9039' '11-9072' '13-1020' '13-1082'
 '13-2020' '13-2051' '13-2054' '15-1252' '17-3019' '17-3028' '19-1099'
 '19-4044' '21-1018' '21-1019' '21-1029' '21-1099' '21-2099' '23-2099'
 '25-1069' '25-1199' '25-2052' '25-3031' '25-3099' '25-9045' '25-9099'
 '27-1019' '27-1029' '27-2091' '27-2099' '27-3099' '27-4015' '27-4099'
 '29-1029' '29-1212' '29-1214' '29-1242' '29-1243' '29-1249' '29-2010'
 '29-2042' '29-2043' '29-2072' '29-9021' '31-1120' '33-1091' '33-1099'
 '33-9094' '35-2019' '35-9099' '37-2019' '37-3019' '39-1014' '39-3019'
 '39-3099' '39-4012' '39-7010' '39-9099' '41-3091' '41-9099' '43-2099'
 '43-3099' '43-4199' '43-9199' '45-2099' '45-4029' '47-3019' '47-4090'
 '47-5049' '47-5099' '49-9069' '51-2028' '51-2090' '51-3099' '51-4199'
 '51-6099' '51-7099' '51-9199' '53-1047' '53-3051' '53-3053' '53-3054'
 '53-3099' '53-4099' '53-6032' '53-6099' '53-7199']
Emp. (millions) in national OES which doesn't have O*NET

In [226]:
low_wfh_cols = [
    'wc_Electronic_Mail_4C1a2h',
    'wc_Outdoors_Exposed_4C2a1c',
    'wc_Outdoors_Under_Co_4C2a1d',
    'wc_Deal_With_Physica_4C1d3',
    'wc_Wear_Common_Prote_4C2e1d',
    'wc_Wear_Specialized_4C2e1e',
    'wc_Exposed_to_Diseas_4C2c1b',
    'wc_Exposed_to_Minor_4C2c1f',
    'wc_Spend_Time_Walkin_4C2d1d',
    'wa_Performing_Genera_4A3a1',
    'wa_Handling_and_Movi_4A3a2',
    'wa_Controlling_Machi_4A3a3',
    'wa_Operating_Vehicle_4A3a4',
    'wa_Performing_for_or_4A4a8',
    'wa_Repairing_and_Mai_4A3b5',
    'wa_Repairing_and_Mai_4A3b4',
    'wa_Inspecting_Equipm_4A1b2'
]

merged_low_wfh = merged[low_wfh_cols].sum(axis=1).copy().to_frame().rename(columns={0: "low_wfh"})
merged_pp = merged["pp"].copy().to_frame()


def weighted_quantile(values, quantiles, sample_weight):
    # sort in ascending order
    sorter = np.argsort(values)
    values = np.array(values)[sorter]
    weights = np.array(sample_weight)[sorter]
    cumulative_weight = np.cumsum(weights) - 0.5 * weights
    cumulative_weight /= cumulative_weight[-1]
    return np.interp(quantiles, cumulative_weight, values)

# # Compute weighted quantiles using TOT_EMP as weights
q1, median, q3 = weighted_quantile(merged_low_wfh['low_wfh'], [0.25, 0.5, 0.75], sample_weight=merged['TOT_EMP'])

merged_low_wfh['low_wfh_binary'] = (merged_low_wfh['low_wfh'] > median).astype(int)
merged_low_wfh['low_wfh_q1']     = (merged_low_wfh['low_wfh'] <= q1).astype(int)
merged_low_wfh['low_wfh_q2']     = ((merged_low_wfh['low_wfh'] <= median) & (merged_low_wfh['low_wfh'] > q1)).astype(int)
merged_low_wfh['low_wfh_q3']     = ((merged_low_wfh['low_wfh'] <= q3) & (merged_low_wfh['low_wfh'] > median)).astype(int)
merged_low_wfh['low_wfh_q4']     = (merged_low_wfh['low_wfh'] > q3).astype(int)

print("LWFH Median =", median)

q1, median, q3 = weighted_quantile(merged['pp'], [0.25, 0.5, 0.75], sample_weight=merged['TOT_EMP'])

merged_pp['high_pp_binary'] = (merged_pp['pp'] > median).astype(int)
merged_pp['pp_q1']     = (merged_pp['pp'] <= q1).astype(int)
merged_pp['pp_q2']     = ((merged_pp['pp'] <= median) & (merged_pp['pp'] > q1)).astype(int)
merged_pp['pp_q3']     = ((merged_pp['pp'] <= q3) & (merged_pp['pp'] > median)).astype(int)
merged_pp['pp_q4']     = (merged_pp['pp'] > q3).astype(int)

print("PP Median =", median)

merged_low_wfh["high_wfh_binary"] = 1 - merged_low_wfh["low_wfh_binary"]
merged_pp["low_pp_binary"]   = 1 - merged_pp["high_pp_binary"]


LWFH Median = 2.0
PP Median = 3.639802223600274


/software/anaconda/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


In [230]:
# Normalize the 'pp' values in merged_pp to [0, 1]
pp_min = merged_pp['pp'].min()
pp_max = merged_pp['pp'].max()
merged_pp['pp'] = (merged_pp['pp'] - pp_min) / (pp_max - pp_min)

# Normalize the 'low_wfh' values in merged_low_wfh to [0, 1]
wfh_min = merged_low_wfh['low_wfh'].min()
wfh_max = merged_low_wfh['low_wfh'].max()
merged_low_wfh['low_wfh'] = (merged_low_wfh['low_wfh'] - wfh_min) / (wfh_max - wfh_min)

# Create the 'high_wfh' column as the complement of 'low_wfh'
merged_low_wfh['high_wfh'] = 1 - merged_low_wfh['low_wfh']

In [241]:
pd.concat([merged_low_wfh, merged_pp], axis=1).reset_index().to_csv("/project/high_tech_ind/WFH/WFH/data/aux_and_croswalks/low_wfh_pp.csv", index=False)